In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./datas/final2019(rev4).csv', index_col=[0])
df.tail()

,Artist,Genre,Lyrics,Title,Year
4253,아이유,록/메탈,‘뭐해?‘라는 두 글자에‘네가 보고 싶어’ 나의 속마음을 담아 우이모티콘 하나하나 ...,Blueming,2019
4254,MC몽,랩/힙합,우선 캐릭터는 분명해야 해요 X 같은 XX철부지같이 굴면Fail 해요 Next 바로...,"인기 (Feat. 송가인, 챈슬러)",2019
4255,ITZY (있지),댄스,Hey hey hey yass whoo Beep beep Hey hey hey he...,ICY,2019
4256,Lil tachi,랩/힙합,멋이 다 흘러흘러서 넘쳐멀어 버렸어배로 막 벌어멋이 다 흘러멋이 다 흘러배로 막 벌...,눈 (Prod. 기리보이),2019
4257,홍진영,성인가요,오늘 밤에 아무도 모르게너랑 둘이서 둘이서 사랑을 할거야오늘 밤에 아무도 모르게뚜뚜...,오늘 밤에,2019


In [ ]:
# 모듈 생성

In [ ]:
### had to manually make folder because of the following error

In [2]:
!rm -rf song
!mkdir song/top_music
!mkdir song/user
!touch song/song.py
!touch song/top_music/details.py
!touch song/user/user_playlist.py

The syntax of the command is incorrect.
The syntax of the command is incorrect.
touch: cannot touch 'song/song.py': No such file or directory
touch: cannot touch 'song/top_music/details.py': No such file or directory
touch: cannot touch 'song/user/user_playlist.py': No such file or directory


In [1]:
!tree song

Folder PATH listing
Volume serial number is F653-291E
C:\USERS\LUVU1\DOCUMENTS\DSS15\CRAWLING_PROJECT\CRAWLING_CODE\SONG
����recomm
����top_music
����user


In [ ]:
# 1. 음악 정보 수집 함수
### Melon TOP100 songs of each year between 1964~2019 (song Title, Artist, Link, Year)

In [3]:
%%writefile song/top_music/details.py

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

def song_details(year):
    url = "https://www.melon.com/chart/age/list.htm?idx=1&chartType=YE&chartGenre=KPOP&chartDate={}&moved=Y".format(year)
    headers = {
        "User-Agent" : UserAgent().chrome
    }
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select("#frm > table > tbody tr")
    
    datas = []
    for element in elements:
        data = {
            "Title" : element.select_one('a > .odd_span').text.split('상')[0].strip(),
            "Artist" : element.select_one('div > .fc_mgray').text,
            "Year" : year,
            "Link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
        }
        datas.append(data)
    df = pd.DataFrame(datas)
    
    return df

#상세 페이지 내 장르 수집 함수
def get_genre(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    
    return dom.select_one('dd:nth-of-type(3)').text

#상세 페이지 내 가사 수집 함수
def get_lyrics(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    elements = dom.select_one('.wrap_lyric').text.replace('\r', '').replace('\n', '').replace('\t', '').split('펼치기')[0]
    if elements == '[가사 준비중] 멜론 회원 여러분! 가사 등록을 기다리고 있어요.가사등록하기':
        elements = '-'
        
    return elements

def run(year):
    df = song_details(year)
    df['Genre'] = df['Link'].apply(get_genre)
    df['Lyrics'] = df['Link'].apply(get_lyrics)
    df.drop(columns=['Link'], inplace=True)
    
    return df

Writing song/top_music/details.py


In [ ]:
# 모듈 작동 확인

In [2]:
import song.libs.details as details

details.run(1964)

,Artist,Title,Year,Genre,Lyrics
0,금호동,내일또 만납시다,1964,성인가요,하루의 일을 끝내고 돌아가는거리엔 사람의 물결하늘엔 별이 하나 둘 반짝이면가로등 하...
1,최숙자,눈물의 연평도,1964,성인가요,조기를 담뿍잡아기폭을 올리고온다던 그배는어이하여 아니오나수평선 바라보며그이름 부르면...
2,이미자,님이라 부르리까,1964,성인가요,<1절 임이라 부르리까 당신이라고 부르리까 사랑을 하면서도 사랑을 참고사는 마음으로...
3,이미자,동백아가씨,1964,성인가요,헤일 수 없이 수많은 밤을내 가슴 도려내는 아픔에 겨워얼마나 울었던가 동백아가씨그리...
4,현미,떠날때는 말없이,1964,성인가요,그 날 밤 그 자리에둘이서 만났을 때똑같은 그 순간에똑같은 마음이달빛에 젖은 채밤새...
5,백야성,마도로스 부기,1964,성인가요,항구의 일~~번지 부기우기 일~번지그라스를 채워~다오 부기우기 아~~가씨고동이 슬피...
6,최희준,맨발의 청춘,1964,성인가요,눈물도 한숨도나 혼자 씹어 삼키며밤 거리의 뒷골목을누비고 다녀도사랑만은 단 하나의목...
7,권혜경,물새우는 해변,1964,성인가요,-
8,박재란,밀짚모자 목장아가씨,1964,성인가요,시원한 밀짚모자포플라 그늘에양떼를 몰고가는목장의 아가씨연분홍 빛 입술에는살며시 웃음...
9,현미,보고싶은 얼굴,1964,성인가요,눈을 감고 걸어도눈을 뜨고 걸어도보이는 것은 초라한 모습보고 싶은 얼굴거리마다 물결...


In [ ]:
# 2. 사용자 플레이리스트 수집 함수
### only melon users' playlist can be gathered

In [4]:
%%writefile song/user/user_playlist.py

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

def playlist(seq):
    url = "https://www.melon.com/mymusic/playlist/mymusicplaylistview_listSong.htm?plylstSeq={}".format(seq)
    headers = {
        "User-Agent" : UserAgent().chrome
    }
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select("#frm > div > table > tbody tr")
    
    datas = []
    for element in elements:
        data = {
            "Title" : element.select_one("a.fc_gray").text,
            "Artist" : element.select_one("#artistName").text.replace('\n', ''),
            "Link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
            }
        datas.append(data)
    df_user = pd.DataFrame(datas)
    
    return df_user

#상세 페이지 내 장르 수집 함수
def get_genre(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    
    return dom.select_one('dd:nth-of-type(3)').text

#상세 페이지 내 가사 수집 함수
def get_lyrics(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    elements = dom.select_one('.wrap_lyric').text.replace('\r', '').replace('\n', '').replace('\t', '').split('펼치기')[0]
    if elements == '[가사 준비중] 멜론 회원 여러분! 가사 등록을 기다리고 있어요.가사등록하기':
        elements = '-'
        
    return elements

def user(seq):
    df_user = playlist(seq)
    df_user['Genre'] = df_user['Link'].apply(get_genre)
    df_user['Lyrics'] = df_user['Link'].apply(get_lyrics)
    df_user.drop(columns=['Link'], inplace=True)
    
    return df_user

Writing song/user/user_playlist.py


In [ ]:
# 모듈 작동 확인

In [3]:
import song.user.user_playlist as playlist

playlist.user(486216850)

,Artist,Title,Genre,Lyrics
0,버즈,나에게로 떠나는 여행,록/메탈,나에게로 떠나는 여행저 푸른 바다 끝까지말을 달리면소금 같은 별이 떠있고사막엔 낙타...
1,EXO,나비소녀 (Don`t Go),발라드,조그만 날갯짓 널 향한 이끌림 나에게 따라오라 손짓한 것 같아서 애절한 눈빛과 무...
2,EXO,피터팬 (Peter Pan),댄스,낡은 일기장 먼지를 털어내 문득 펼친 곳 그 속엔 해맑게네가 있어 아직 넌 그대로 ...
3,EXO,"Baby, Don`t Cry (인어의 눈물)",발라드,더는 망설이지 마 제발 내 심장을 거두어 가 그래 날카로울수록 좋아 달빛 조차도 눈...
4,어반자카파,코끝에 겨울,발라드,점차 나아지겠지 혼자 내렸던 그 모든 결론이 마치 먼지처럼 흩어져 사라져 버리던 날...
5,어반자카파,거꾸로 걷는다,발라드,돌아서기 아쉬워 거꾸로 걷는다 끝을 아는 내 발길 거꾸로 걷는다 거꾸로 걷는다 거꾸...
6,어반자카파,서울 밤 (feat. 빈지노),R&B;/Soul,서울의 밤 먼지를 감췄지골목골목 걸으며 Baby talk to meWalk and ...
7,어반자카파,"그때의 나, 그때의 우리",R&B;/Soul,그때의 나 그때의 우리참 어리석고 어렸지그때의 우리 아무것도 아닌 일에다투던 초라할...
8,빈지노 (Beenzino),Boogie On & On,랩/힙합,난 괜찮아내일 아침에 소파 위에 시체가되어 이름 모르는 너와 어색한사이가 되어도I ...
9,어반자카파,목요일 밤 (Feat. 빈지노),R&B;/Soul,평범한 목요일 밤널 데려 갈게 어디든일주일 중에 네가제일 지쳐 있을 오늘Wanna ...


In [4]:
!mkdir song/recomm
!tree song

song
├── recomm
├── song.py
├── top_music
│   └── details.py
└── user
    ├── __pycache__
    │   └── user_playlist.cpython-36.pyc
    └── user_playlist.py

4 directories, 4 files


In [ ]:
# 3. 가사 분석 후 사용자가 입력한 단어와 유사한 음악 추천 함수

In [1]:
%%writefile song/recomm/lyrics_analysis.py

from konlpy.tag import Okt
t = Okt()
import re
import gensim
from gensim.models import word2vec
import pandas as pd
import pymongo

client = pymongo.MongoClient('mongodb://dss:dss@'ip':27017')
db = client['melon_chart']
collection = db.song_list
df = pd.DataFrame(list(collection.find()))
df.drop(columns=['_id'], inplace=True)

# 가사 전처리: 한글, 영어, 숫자만 남기고 모든 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

# 기간, 연관 단어 입력 하여 유사한 음악 추첯
def search_similarity(year_end, year_start, look_keyword):
    yrange = df[(df['Year'] < year_end) & (df['Year'] >= year_start)].reset_index(drop=True)
    lyrics = yrange['Lyrics'].apply(cleanse)
    lyrics = lyrics.tolist()
        
    results = []
    for i in lyrics:
        result = t.pos(i, norm=True, stem=True)
        
        word = []
        for a in result:
            if not a[1] in ["Josa", "Determiner", "Exclamation", "Suffix"]:
                word.append(a[0])
        words = (" ".join(word)).strip()
        results.append(words)
        
    data_file = 'lyrics.data'
    with open(data_file, 'w', encoding='utf-8') as fp:
        fp.write("\n".join(results))
    data = word2vec.LineSentence(data_file)
    model = word2vec.Word2Vec(data, size=200, window=10, hs=1, min_count=2, sg=1)
    model.save('lyrics.model')
    model = word2vec.Word2Vec.load("lyrics.model")
    song = model.wv.most_similar(look_keyword, topn=10)
    keyword = []
    for s in song:
        keyword.append(s[0])
    relist = yrange[yrange['Lyrics'].str.contains('|'.join(keyword))]
    
    return relist.sample(n=10)

Overwriting song/recomm/lyrics_analysis.py


In [ ]:
# 모듈 작동 확인

In [5]:
import song.recomm.lyrics_analysis as search

dj = search.search_similarity(2015, 2010, '사랑')

In [101]:
dj.to_dict(orient="recode")

[{'Artist': '디셈버',
  'Genre': '발라드',
  'Lyrics': "내 사랑은 변하지 않을께저 하늘은 내 맘 알고있겠죠난 언제나 하늘 높은곳에서 그대를비춰주는 별이될께널 사랑한다는 말은참 아껴왔는데그댄 하늘 저멀리 날 두고가네요늘 언제나 니곁을 지켜주지못해내가 정말 미안해요I'M Still With You우리함께 걷던 이길도어느새 이렇게도 많이 변했는데오 오 난 아직도 잊지 못하네요비록 사랑이 눈물일 지라도내 사랑은 변하지 않을께저 하늘은 내 맘 알고있겠죠난 언제나 하늘 높은곳에서 그대를비춰주는 별이될께그대는 목소리도 참 고왔었는데이세상 어디에도 그대는 없네요늘 어린아이처럼매일 투정 부린 내가정말 후회가 되요I'M Still With You우리함께 나눈 추억도가슴에 내 머리속엔 다 그대론데오 오 난 아직도 잊지 못하네요비록 사랑이 눈물일 지라도내 사랑은 변하지 않을께저 하늘은 내 맘 알고있겠죠난 언제나 하늘 높은곳에서 그대를비춰주는 별이될께내 사랑을 꼭 기억해 줘요내 눈물을 그댄 듣고있나요기억할께 십년이 지나도 나 약속해그대를 잊지 않을께그댈 비추는 별이될께 ",
  'Title': '별이될께',
  'Year': 2010},
 {'Artist': '성시경',
  'Genre': '발라드',
  'Lyrics': '그대네요 정말 그대네요그 따뜻한 눈빛은 늘 여전하네요이제야 날 봤나요한참을 보고 있었는데햇살이 어루만지는 그대 얼굴꿈일지도 몰라안녕이란 말도 나오질 않아하고픈 말 얼마나 많았는데꿈에서도 너만 찾았는데너무 늦었단 그런 말은 듣기 싫은데내 눈을 피하는 그대내 맘이 보이나요그대 눈빛을 난 알 수가 없어소리 없이 나 혼자 안녕알고 있죠지나간 일에는너무 힘들었죠만약이란 없는 것 그래도 한번쯤달려가 잡았다면그대가 한번만 안아줬더라면그렇게 돌아서진 않았을텐데꿈일지도 몰라안녕이란 말도 나오질 않아하고픈 말 얼마나 많았는데꿈에서도 너만 찾았는데혹시 그대도 하루마다 힘들었다면나를 그리워했다면주저하지 말아요다시 망설이다 놓칠 순 없어떨리는 목소리로

In [74]:
dj1 = dj[['Artist', 'Title', 'Genre']].values
dj1

array([['디셈버', '별이될께', '발라드'],
       ['성시경', '그대네요', '발라드'],
       ['허각', '짧은머리', '발라드'],
       ['양요섭', 'LOVE DAY', '댄스'],
       ['씨스타19', 'Ma Boy (마보이)', '댄스']], dtype=object)

In [ ]:
# 4. 음악 데이터와 사용자 플레이리스트간 장르 유사도 측정 후 음악 추천 함수

In [2]:
%%writefile song/recomm/ranking_random.py

import pandas as pd
import pymongo
import math
import numpy as np
import random
import song.user.user_playlist as playlist

# mongodb에 연결해서 dataframe 만들기 : 전체곡 정보 데이터프레임 : df
client = pymongo.MongoClient('mongodb://dss:dss@'ip':27017')
db = client['melon_chart']
collection = db.song_list
df = pd.DataFrame(list(collection.find()))
df.drop(columns=['_id'], inplace=True)


# 장르선호도가 유사한 연도 ranking top5 뽑는 함수
def top5_list(seq, df=df):
    # NEW DATAFRAME
    recommend = []
    recommend_df = pd.DataFrame()
    
    # 1. 전체인기곡들 장르유사도
    df = df.reset_index(drop = True)
    df["count"] = 1
    df_pivot = pd.pivot_table(data = df , values = "count", index = "Year", columns = "Genre",aggfunc = np.sum)
    df_pivot.fillna(0, inplace = True)
    df_normal = round(df_pivot.div(df_pivot.sum(axis = 1), axis=0),3)
    df_normal = df_normal.reset_index()
    
    # 2. 유저플레이리스트 장르유사도  함수
    df_user = playlist.user(seq)
    df_user.reset_index()
    # PIVOT
    df_user["count"] = 1
    user_pivot = pd.pivot_table(data = df_user, values = "count", columns="Genre",  aggfunc = np.sum)
    user_pivot.fillna(0, inplace=True)
    # 특정장르곡수/전체장르곡수 나눈 값 df : 사용자플레이리스트의 장르유사도 df
    user_normal = round(user_pivot.div(user_pivot.sum(axis=1), axis=0), 3)
    user_normal = pd.DataFrame(user_normal, columns=['R&B/Soul', '국내영화/국내드라마', '그외', '댄스','랩/힙합','록/메탈','발라드','성인가요','인디음악','재즈','포크/블루스'])
    np.nan_to_num(user_normal, copy=False)

    # 3. 장르유사도 비교 : 전체인기곡들 & 유저플레이리스트
    # 오차 구하기
    for i in range(len(df_normal)):
        error = (df_normal.loc[i] - user_normal) ** 2
        error = error.sum(axis =1)
        error = np.sqrt(error)
        recommend.append(error)
        
    # rms 칼럼만 만들기, 컬럼명 count를 rms로 변경
    from pandas import DataFrame
    recommend_df = pd.DataFrame(recommend)
    recommend_df.rename(columns={"count":"rms"}, inplace=True)
    
    # recommend_df 에 Year 값 넣은 df 만들기, df_normal에서 Year 컬럼 선택
    year = pd.DataFrame(df_normal["Year"])
    
    # Year 컬럼있는 df와 rms 컬럼있는 df 합치기
    recom = pd.concat([year,recommend_df], axis=1, join="inner")
    
    # 유사도 상위5개 : 인덱스 수정하기 위해 .reset_index(drop=True)
    ranking = recom.sort_values(by="rms", ascending=True).reset_index(drop=True)
    
    # 상위 5개 뽑기
    top5_r = pd.DataFrame()
    for i in range(0, 5):
        top5 = df[df["Year"] == ranking["Year"][i]]
        top5_r = pd.concat([top5_r, top5])

    fir = df[df["Year"] == ranking["Year"][0]] 
    sec = df[df["Year"] == ranking["Year"][1]]
    thr = df[df["Year"] == ranking["Year"][2]]
    four = df[df["Year"] == ranking["Year"][3]]
    fif = df[df["Year"] == ranking["Year"][4]]
    
    # 상위 5개 연도에서 가중치부여해서 random 곡 뽑기
    first = fir.take(np.random.permutation(len(fir))[:10])
    second = sec.take(np.random.permutation(len(fir))[:8])
    third = thr.take(np.random.permutation(len(fir))[:6])
    fourth = four.take(np.random.permutation(len(fir))[:4])
    fifth = fif.take(np.random.permutation(len(fir))[:2])
    
    # 뽑은 곡들 합해서 새로운 playlist 만들기
    result = pd.concat([first, second, third, fourth, fifth])
    
    # 곡 정보 중에서 가수명,곡명,장르명 만 보여주기
    playlist30 = result.loc[:, ["Artist", "Title", "Genre"]]
    
    return playlist30

Overwriting song/recomm/ranking_random.py


In [3]:
webhook = "SLACK_WEBHOOK"

In [ ]:
# 8. 슬랙 전송
### Sending to Slack channel through webhook 

In [1]:
%%writefile song/user/send_message.py

import requests
import json

def send_msg(webhook, dj, channel='#dss', username='DJ'):
    dj = str(dj[['Artist', 'Title', 'Genre']].values)
    payload = {"channel": channel, "username": username, "text": dj}
    requests.post(webhook, data=json.dumps(payload))

Overwriting song/user/slack.py


In [106]:
def send_msg(webhook, dj, channel='#dss', username='DJ'):
    dj = str(dj[['Artist', 'Title', 'Genre']].values)
    payload = {"channel": channel, "username": username, "text": dj}
    requests.post(webhook, data=json.dumps(payload))

In [107]:
send_msg(webhook, dj)

In [ ]:
# 모듈 작동 확인

In [6]:
import song.user.slack as slack

slack.send_msg(webhook, dj)